In [ ]:
using CairoMakie
using Arya
using LilGuys


In [ ]:
function read_distribution(name)
    return LilGuys.read_fits(joinpath(name, "peris_apos.fits"))
end

In [ ]:
function compare_peris(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "pericentre / kpc",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.pericenter, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
function compare_apos(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "apocentre / kpc",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.apocenter, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
using Printf

In [ ]:
using StatsBase

In [ ]:
function compare_stats(families)

    for (label, df) in families
        @printf "%16s%12.2f [%0.2f %0.2f] [[%0.2f %0.2f]]\n" label median(df.pericenter) quantile(df.pericenter, 0.16) quantile(df.pericenter, 0.84) quantile(df.pericenter, 0.001) quantile(df.pericenter, 0.999)
    end

end

In [ ]:
compare_stats(families)

In [ ]:
families = [
    "strict" => read_distribution("reported_uncertanties"),
    "+gaia sys" => read_distribution("systematic_errors"),
    "+solar lsr sys" => read_distribution("sys_and_solar"),
    "all studies" => read_distribution("all_studies"),
    "iorio" => read_distribution("iorio")
    ]

In [ ]:
compare_peris(families)

In [ ]:
compare_apos(families)